# Insertion de nouveaux points

In [1]:
import os
import glob

directory = "results/assets"
notebook_path = os.path.abspath("Notebook.ipynb")
chemin = os.path.join(os.path.dirname(notebook_path), directory)
paths = glob.glob(chemin+'/*')

print("done")

done


In [2]:
from insertion_data import *

client = pymongo.MongoClient('localhost',27017)
collection = client.fcd.assets

print('done')

done


In [ ]:
insert_data(paths, "assets", collection)


file 14256004530491 ...
file 14256004530491 done, 1 assets added
file 14256004535292 ...
file 14256004535292 done, 2 assets added
file 14256004559268 ...
file 14256004559268 done, 3 assets added
file 14256004569226 ...
file 14256004569226 done, 4 assets added
file 14256004574044 ...
file 14256004574044 done, 5 assets added
file 14256004592574 ...
file 14256004592574 done, 6 assets added
file 14256004644482 ...
file 14256004644482 done, 7 assets added
file 14256004664720 ...
file 14256004664720 done, 8 assets added
file 14295000005090 ...
file 14295000005090 done, 9 assets added
file 14295000006551 ...


In [4]:
# ajout index géo spatial
add_2dsphere_index(collection, "location.geo")

index response: location.geo_2dsphere
